Se tiene una colección de documentos (textos) y se quiere
calcular el promedio de la cantidad de caracteres que tienen
las frases de 2,3 y 4 palabras en toda la colección. Programar
la solución a este problema usando Map-Reduce usando
combiners como método de agregación. 
[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqsnl524yi)

Voy a definir una funcion que dado un documento devuelva una lista de frases de 2, 3 y 4 palabras. Luego mapearla a un RDD con un par de documentos de wikipedia.

In [ ]:
def get_frases(documento):
    frases = []
    for frase in documento.replace('\n',' ').split("."):
        palabras = frase.split(" ")
        for i in range(len(palabras)+1):
            if i>=2: frases.append((" ".join(palabras[i-2:i]),2))
            if i>=3: frases.append((" ".join(palabras[i-3:i]),3))
            if i>=4: frases.append((" ".join(palabras[i-4:i]),4))
    return frases

In [ ]:
import wikipedia as wp
wp.set_lang("en")
articulo1 = wp.page("Paul Gilbert").summary
articulo2 = wp.page("Chris Impellitteri").summary
articulo3 = wp.page("Steve Vai").summary
articulo4 = wp.page("Eddie Van Halen").summary
rdd = sc.parallelize([articulo1,articulo2,articulo3,articulo4])

Tenemos un RDD donde cada registro es el resumen de una pagina de wikipedia. Mapeamos get_frases a cada registro.

In [ ]:
rdd = rdd.map(get_frases).flatMap(lambda x: x)

Ahora tenemos un RDD donde cada registro es una lista de tuplas de la forma `<palabras, cantidad de palabras>`. La idea seria agrupar por cantidad de palabras en la frase. Para conocer el promedio de caracteres, necesitamos la cantidad de frases y la suma de sus largos. Entonces necesitamos sumar los largos y 1 por cada palabra. 
Acomodamos la cantidad de palabras en la frase como clave y hacemos `reduceByKey` para sumar.

In [ ]:
rdd = rdd.map(lambda x: (x[1], (len(x[0]), 1)))
rdd = rdd.reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1]))

Teniendo la cantidad de frases y la suma de caracteres, hacemos map sobre los valores con `mapValues`.

In [ ]:
rdd = rdd.mapValues(lambda x: round(float(x[0])/x[1],2))

In [ ]:
rdd.collect()